# Electricity Generation from api.electricitymap.org
---

# This is the file in which I'm seeking to create the dataframe without the extreme fragmenting.  I need to get the json data into series form, I think.

The Elec_Generation_USstable.ipynb has the working code.

SECTION 1 
collect data on carbon intensity, zones within SW , and power breakdown from the electricitymap api

In [1]:
# Dependencies and Setup

import pandas as pd
import requests


In [2]:

# zones for electrical utilities in US
zones = ["US-SW-PNM", "US-SW-EPE", "US-SW-WALC", "US-NW-PACE", "US-NW-PSCO", "US-CENT-SWPP", "US-TEX-ERCO", "US-MIDW-AECI","US-SW-AZPS","US-SW-AZPS",
         "US-NW-WACM", "US-SW-SRP", "US-SW-TEPC", "US-CENT-SPA", "US-CAL-IID", "US-CAL-CISO", "US-CAL-BANC","US-CAL-BANC", "US-CAL-TIDC", 
          "US-CAR-CPLE", "US-CAR-CPLW", "US-CAR-DUK", "US-CAR-SC", "US-CAR-SCEG", "US-CAR-YAD", "US-FLA-FMPP", "US-FLA-FPC" , "US-FLA-FPL",
          "US-FLA-GVL" , "US-FLA-HST", "US-FLA-JEA", "US-FLA-SEC", "US-FLA-TAL", "US-FLA-TEC", "US-MIDW-AECI" , "US-MIDW-LGEE", "US-MIDW-MISO",
          "US-NE-ISNE", "US-NW-BPAT", "US-NW-CHPD", "US-NW-DOPD", "US-NW-GCPD", "US-NW-GRID",  "US-NW-IPCO" , "US-NW-NWMT", "US-NW-NEVP", 
           "US-NW-PACW",  "US-NW-PGE", "US-NW-PSEI", "US-NW-SCL", "US-NW-TPWR", "US-NW-WAUW", "US-NY-NYIS", "US-SE-SEPA", "US-SE-SOCO" , 
           "US-TEN-TVA"]


In [3]:

# get carbon intensity history for the US utilities
urls = []
for index, url in enumerate(zones):
    url = f'https://api.electricitymap.org/v3/carbon-intensity/history?zone={zones[index]}'
    urls.append(url)

# store responses in responses_dict dictionary
responses_dict = {}
for idx, url in enumerate(urls):
    response = requests.get(url)
    responses_dict[f"response_{idx+1}"] = response.json()


In [4]:
#request power breakdown
pburls = []
for index, url in enumerate(zones):
    pburl = f'https://api.electricitymap.org/v3/power-breakdown/history?zone={zones[index]}'
    pburls.append(pburl)

#store response in power_breakdown_responses_dict dictionary
power_breakdown_responses_dict = {}
for idx, pburl in enumerate(pburls):
    response = requests.get(pburl)
    power_breakdown_responses_dict[f"response_{idx+1}"] = response.json()


SECTION 2     
PowerBreakdown data transformation

In [6]:
# pull data from power breakdown json in dataframe
region = power_breakdown_responses_dict['response_1']['history'][0]['zone']
datetime = power_breakdown_responses_dict['response_1']['history'][0]['datetime']
nuclear = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['nuclear']
geothermal = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['geothermal']
biomass = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['biomass']
coal = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['coal']
wind = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['wind']
solar = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['solar']
hydro = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro']
gas = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['gas']
oil = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['oil']
unknown = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['unknown']
hydro_discharge = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro discharge']
battery_discharge = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['battery discharge']
renewable_percentage = power_breakdown_responses_dict['response_1']['history'][0]["renewablePercentage"]
total_consumption = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionTotal"]
estimated = power_breakdown_responses_dict['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
df_US = pd.DataFrame({'region':[region],'datetime':[datetime],'nuclear':[nuclear],'geothermal':[geothermal],'biomass':[biomass], 'coal':[coal], 'wind':[wind], 'solar':[solar], 
           'hydro':[hydro], 'gas':[gas], 'oil':[oil], 'unknown':[unknown], 'hydro-discharge':[hydro_discharge], 
           'battery_discharge':[battery_discharge], 'renewable_percentage':[renewable_percentage], 'total_consumption':[total_consumption], 
           'estimated':[estimated]})

# Create a dataFrame with the first values
#df_US = pd.DataFrame.from_dict(us_pnm1,orient='index')


In [16]:
# Data wrangling from the response to create a legible dataFrame
# outer for loop for regions/responses
l_us = []
for reg in range(len(zones)):
    #for each zone
    response = f"response_{reg+1}"
   
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone

        dict_us =  {'region'[0]:power_breakdown_responses_dict['response_1']['history'][0]['zone'], 
                        'datetime':power_breakdown_responses_dict['response_1']['history'][0]['datetime'],
                        'nuclear':power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['nuclear'],
                        'geothermal':power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['geothermal'],
                        'biomass':power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['biomass'],
                        'coal':power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['coal'], 
                        'wind':power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['wind'], 
                        'solar':power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['solar'], 
                        'hydro':power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro'],
                        'gas':power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['gas'],
                        'oil':power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['oil'],
                        'unknown':power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['unknown'],
                        'hydro-discharge':power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro discharge'], 
                        'battery_discharge':power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['battery discharge'],
                        'renewable_percentage':power_breakdown_responses_dict['response_1']['history'][0]["renewablePercentage"],
                        'total_consumption':power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionTotal"], 
                        'estimated':power_breakdown_responses_dict['response_1']['history'][0]["isEstimated"]}
        l_us.append(dict_us)
        
        if i == 0 & reg == 0:
            df_US = pd.DataFrame(l_us)
        else:
            df_new = pd.DataFrame(l_us)
            df_US = pd.concat([df_US, df_new])


In [18]:
df_US.describe()

,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption
count,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0
mean,0.0,0.0,0.0,469.0,1382.0,2.0,18.0,723.0,0.0,141.0,0.0,0.0,51.0,2735.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,469.0,1382.0,2.0,18.0,723.0,0.0,141.0,0.0,0.0,51.0,2735.0
25%,0.0,0.0,0.0,469.0,1382.0,2.0,18.0,723.0,0.0,141.0,0.0,0.0,51.0,2735.0
50%,0.0,0.0,0.0,469.0,1382.0,2.0,18.0,723.0,0.0,141.0,0.0,0.0,51.0,2735.0
75%,0.0,0.0,0.0,469.0,1382.0,2.0,18.0,723.0,0.0,141.0,0.0,0.0,51.0,2735.0
max,0.0,0.0,0.0,469.0,1382.0,2.0,18.0,723.0,0.0,141.0,0.0,0.0,51.0,2735.0


In [19]:
# fill NA values with zeroes for energy values
df_US = df_US.fillna({'nuclear': 0,'geothermal': 0,'biomass': 0, 'coal': 0, 'wind': 0, 'solar': 0, 
           'hydro': 0, 'gas': 0, 'oil': 0, 'unknown': 0, 'hydro-discharge':0, 
           'battery_discharge':0})
df_US.describe()

,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption
count,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0,31980.0
mean,0.0,0.0,0.0,469.0,1382.0,2.0,18.0,723.0,0.0,141.0,0.0,0.0,51.0,2735.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,469.0,1382.0,2.0,18.0,723.0,0.0,141.0,0.0,0.0,51.0,2735.0
25%,0.0,0.0,0.0,469.0,1382.0,2.0,18.0,723.0,0.0,141.0,0.0,0.0,51.0,2735.0
50%,0.0,0.0,0.0,469.0,1382.0,2.0,18.0,723.0,0.0,141.0,0.0,0.0,51.0,2735.0
75%,0.0,0.0,0.0,469.0,1382.0,2.0,18.0,723.0,0.0,141.0,0.0,0.0,51.0,2735.0
max,0.0,0.0,0.0,469.0,1382.0,2.0,18.0,723.0,0.0,141.0,0.0,0.0,51.0,2735.0


In [21]:
# convert measured Energy values to integers in Giga Watts
convert_dict = {'hydro': int}
#convert_dict = {'nuclear': int, 'geothermal': int, 'biomass': int, 'coal': int, 'wind': int, 'solar': int, 'hydro': int, 'gas': int, 'oil': int, 
             #   'hydro-discharge': int, 'battery_discharge': int, 'renewable_percentage': int, 'total_consumption': int
               # }
 # note - the unknown column only has values rarely - converting null values to integer doesn't work so this is left as an object
df_US_new = df_US.astype(convert_dict)

#check that data types are changed to int
df_US_new.dtypes

r                       object
datetime                object
nuclear                  int64
geothermal               int64
biomass                  int64
coal                     int64
wind                     int64
solar                    int64
hydro                    int32
gas                      int64
oil                      int64
unknown                  int64
hydro-discharge          int64
battery_discharge        int64
renewable_percentage     int64
total_consumption        int64
estimated                 bool
dtype: object

In [22]:
# Date Time work

# import datetime dependencies

from datetime import datetime

# set up lists to hold parsed data and DateTime as a datetime datetype
dates=[]
times = []
DateTime =[]

# convert date time strings
for i in range(len(df_US_new['datetime'])):

    # Parse the timestamp string to a datetime object
    dt_obj = datetime.strptime(df_US_new.iloc[i,1], '%Y-%m-%dT%H:%M:%S.%fZ')

    date = dt_obj.strftime('%Y-%m-%d')
    time = dt_obj.strftime('%H:%M:%S')

#add the new times and dates to lists

    dates.append(date)
    times.append(time)
    DateTime.append(dt_obj)

# add the times and dates to new columns in the data frame
df_US_new['UTC time'] = times
df_US_new['UTC date'] = dates
df_US_new['UTC DateTime'] = DateTime

In [23]:
#set the UTC DateTime as the index
df_US_new_reindex = df_US_new.set_index('UTC DateTime', inplace=True)

#drop the datetime column that contains a string
df_US_newer = df_US_new.drop('datetime', axis=1)
df_US_newer.head()

,r,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated,UTC time,UTC date
UTC DateTime,,,,,,,,,,,,,,,,,,
2024-07-21 02:00:00,US-SW-PNM,0,0,0,469,1382,2,18,723,0,141,0,0,51,2735,False,02:00:00,2024-07-21
2024-07-21 02:00:00,US-SW-PNM,0,0,0,469,1382,2,18,723,0,141,0,0,51,2735,False,02:00:00,2024-07-21
2024-07-21 02:00:00,US-SW-PNM,0,0,0,469,1382,2,18,723,0,141,0,0,51,2735,False,02:00:00,2024-07-21
2024-07-21 02:00:00,US-SW-PNM,0,0,0,469,1382,2,18,723,0,141,0,0,51,2735,False,02:00:00,2024-07-21
2024-07-21 02:00:00,US-SW-PNM,0,0,0,469,1382,2,18,723,0,141,0,0,51,2735,False,02:00:00,2024-07-21


Section 3
Transform carbon intensity data

In [13]:
# pull data from C intensity json in dataframe
region = responses_dict['response_1']['history'][0]['zone']
datetime = responses_dict['response_1']['history'][0]['datetime']
carbon_Intensity = responses_dict['response_1']['history'][0]["carbonIntensity"]
estimated = responses_dict['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1C = {'region':region,'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# Create a dataFrame with the first values
df_US_C = pd.DataFrame.from_dict(us_pnm1C,orient='index')

In [14]:
# Data wrangling from the response to create a legible dataFrame for carbon intensity history

# outer for loop for regions/responses
for reg in range(len(zones)):
    response = f"response_{reg+1}"
    
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = responses_dict['response_1']['history'][0]['zone']
        datetime = responses_dict['response_1']['history'][0]['datetime']
        carbon_Intensity = responses_dict['response_1']['history'][0]["carbonIntensity"]
        estimated = responses_dict['response_1']['history'][0]["isEstimated"]
        
        # this 24 is also for the 24 hours of data in each zone
        df_US_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# make the datetime the rows and carbon_intensity a column
df_US_C_new = df_US_C.transpose()

# check data types
df_US_C_new.dtypes

C:\Users\mrkol\AppData\Local\Temp\ipykernel_36944\3243690213.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_US_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}


region              object
datetime            object
Carbon_Intensity    object
estimated           object
dtype: object

In [15]:
# convert carbon intensity measurement to an integer in g CO2e/kWh
convert_dict_C= {'Carbon_Intensity': int}
 
df_US_C_new = df_US_C_new.astype(convert_dict_C)

#check that the datatype has been changed
df_US_C_new.dtypes

region              object
datetime            object
Carbon_Intensity     int32
estimated           object
dtype: object

In [16]:
# add the times and dates to new columns in the data frame   -    This assumes the data for carbon intensity is pulled at the same time as power breakdown
df_US_C_new['UTC time'] = times
df_US_C_new['UTC date'] = dates
df_US_C_new['UTC DateTime'] = DateTime

#set the UTC DateTime as the index
df_US_C_new_reindex = df_US_C_new.set_index('UTC DateTime', inplace=True)
#drop the datetime column that contains a string
df_US_C_newer = df_US_C_new.drop('datetime', axis=1)

Section 4
Merge dataframes

In [17]:
df_power_and_carbon= pd.merge(df_US_newer, df_US_C_newer,on=['UTC DateTime','region','UTC time','UTC date'])



df_power_and_carbon.rename(columns={'Carbon_Intensity':'Carbon_Intensity(gCO2eq/kWh)','total_consumption':'total_consumption(GW)', 'nuclear':'nuclear(GW)', 
                                    'geothermal':'geothermal(GW)', 'biomass':'biomass(GW)', 'coal':'coal(GW)', 'wind':'wind(GW)', 'solar':'solar(GW)', 
                                    'hydro':'hydro(GW)','gas':'gas(GW)', 'region_x': 'region', 'estimated_x': 'breakdown estimated?','estimated_y':'intensity estimated?'}, inplace=True)




df_power_and_carbon.head()

,region,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),breakdown estimated?,UTC time,UTC date,Carbon_Intensity(gCO2eq/kWh),intensity estimated?
UTC DateTime,,,,,,,,,,,,,,,,,,,,
2024-07-20 18:00:00,US-SW-PNM,0,0,0,236,4,934,18,513,0,0,0,0,56,1705,False,18:00:00,2024-07-20,349,False
2024-07-20 18:00:00,US-SW-PNM,0,0,0,236,4,934,18,513,0,0,0,0,56,1705,False,18:00:00,2024-07-20,349,False
2024-07-20 18:00:00,US-SW-PNM,0,0,0,236,4,934,18,513,0,0,0,0,56,1705,False,18:00:00,2024-07-20,349,False
2024-07-20 18:00:00,US-SW-PNM,0,0,0,236,4,934,18,513,0,0,0,0,56,1705,False,18:00:00,2024-07-20,349,False
2024-07-20 18:00:00,US-SW-PNM,0,0,0,236,4,934,18,513,0,0,0,0,56,1705,False,18:00:00,2024-07-20,349,False


In [18]:
# import previous cleaned file into a pandas dataframe
df_us_energy = pd.read_csv('data/runningUSenergy_data.csv')
df_us_energy_reindex=df_us_energy.set_index("UTC DateTime")
df_us_energy_reindex.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,12317.000000,12317.00000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12257.000000,12274.000000,12317.000000
mean,1229.133799,15.10246,23.796704,1731.420476,620.345052,524.483884,654.829260,4170.268897,6.950719,98.859544,0.734919,20.240481,32.360202,9128.080740,407.103191
std,2557.459988,105.61519,110.347903,4564.582186,2141.936954,2053.627946,1506.961363,8140.190461,54.904280,208.223466,15.657951,292.551278,34.402614,16725.758723,170.267252
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,238.000000,0.000000,0.000000,0.000000,0.000000,5.000000,909.000000,266.000000
50%,0.000000,0.00000,0.000000,38.000000,0.000000,0.000000,75.000000,991.000000,0.000000,9.000000,0.000000,0.000000,19.000000,2391.000000,382.000000
75%,736.000000,0.00000,0.000000,1368.000000,195.000000,190.000000,712.000000,3314.000000,0.000000,95.000000,0.000000,0.000000,49.000000,9230.250000,556.000000
max,13803.000000,812.00000,657.000000,49357.000000,21642.000000,19768.000000,65867.000000,54713.000000,2054.000000,2053.000000,557.000000,7075.000000,100.000000,117668.000000,801.000000


In [19]:
df_us_energy_reindex.drop_duplicates(inplace=True)
df_us_energy_reindex.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,12317.000000,12317.00000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12257.000000,12274.000000,12317.000000
mean,1229.133799,15.10246,23.796704,1731.420476,620.345052,524.483884,654.829260,4170.268897,6.950719,98.859544,0.734919,20.240481,32.360202,9128.080740,407.103191
std,2557.459988,105.61519,110.347903,4564.582186,2141.936954,2053.627946,1506.961363,8140.190461,54.904280,208.223466,15.657951,292.551278,34.402614,16725.758723,170.267252
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,238.000000,0.000000,0.000000,0.000000,0.000000,5.000000,909.000000,266.000000
50%,0.000000,0.00000,0.000000,38.000000,0.000000,0.000000,75.000000,991.000000,0.000000,9.000000,0.000000,0.000000,19.000000,2391.000000,382.000000
75%,736.000000,0.00000,0.000000,1368.000000,195.000000,190.000000,712.000000,3314.000000,0.000000,95.000000,0.000000,0.000000,49.000000,9230.250000,556.000000
max,13803.000000,812.00000,657.000000,49357.000000,21642.000000,19768.000000,65867.000000,54713.000000,2054.000000,2053.000000,557.000000,7075.000000,100.000000,117668.000000,801.000000


In [20]:
# concatentate current data with existing file
df_both = pd.concat([df_us_energy_reindex,df_power_and_carbon])

# drop duplicate rows
df_both.drop_duplicates(inplace=True)
df_both.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,Carbon_Intensity(gCO2eq/kWh)
count,12317.000000,12317.00000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000,12317.000000
mean,1229.133799,15.10246,23.796704,1731.420476,620.345052,524.483884,654.829260,4170.268897,6.950719,98.859544,0.734919,20.240481,407.103191
std,2557.459988,105.61519,110.347903,4564.582186,2141.936954,2053.627946,1506.961363,8140.190461,54.904280,208.223466,15.657951,292.551278,170.267252
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,238.000000,0.000000,0.000000,0.000000,0.000000,266.000000
50%,0.000000,0.00000,0.000000,38.000000,0.000000,0.000000,75.000000,991.000000,0.000000,9.000000,0.000000,0.000000,382.000000
75%,736.000000,0.00000,0.000000,1368.000000,195.000000,190.000000,712.000000,3314.000000,0.000000,95.000000,0.000000,0.000000,556.000000
max,13803.000000,812.00000,657.000000,49357.000000,21642.000000,19768.000000,65867.000000,54713.000000,2054.000000,2053.000000,557.000000,7075.000000,801.000000


In [21]:
#df_both.to_csv(f'data/{dates[0]}USenergy_data2.csv')
df_both.to_csv(f'data/runningUSenergy_data.csv')

In [22]:
df_both_cleaned = df_both.loc[df_both['breakdown estimated?']==False,:]
df_both_cleaned.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,Carbon_Intensity(gCO2eq/kWh)
count,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000
mean,1347.987431,40.813528,61.816956,1887.266910,998.741316,700.736517,1034.737203,5243.010512,14.284735,116.933729,0.000457,55.288620,390.085923
std,2326.880108,170.624401,170.947332,4604.869648,2718.630504,2622.954750,1929.009755,8370.025906,83.204524,231.963593,0.021376,484.953484,141.891529
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,270.000000,0.000000,0.000000,0.000000,0.000000,283.000000
50%,0.000000,0.000000,0.000000,32.000000,28.000000,1.000000,216.000000,1366.000000,0.000000,13.000000,0.000000,0.000000,375.000000
75%,2214.000000,0.000000,0.000000,1285.000000,419.250000,224.000000,1294.250000,7948.000000,0.000000,126.250000,0.000000,0.000000,460.000000
max,13287.000000,812.000000,657.000000,48997.000000,21642.000000,19768.000000,65867.000000,54713.000000,2054.000000,2053.000000,1.000000,7075.000000,801.000000


In [23]:
df_both_cleaned.to_csv(f'data/runningUSenergy_data_filtered.csv')